In [2]:
%pip uninstall numpy -y

Found existing installation: numpy 2.1.2
Uninstalling numpy-2.1.2:
  Successfully uninstalled numpy-2.1.2
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip show numpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install numpy==1.24.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 161.2 MB/s  0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.24.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import mmdet
import mmcv
import torch

print(f"MMDetection: {mmdet.__version__}")
print(f"MMCV: {mmcv.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


MMDetection: 3.3.0
MMCV: 2.1.0
CUDA available: True


In [6]:
import zipfile
import os

# Set paths
zip_path = "/home/ronan/dataset/car_damages_new_v2.v1i.coco-mmdetection.zip"
extract_dir = "/home/ronan/dataset/car_damages_new_v2/"

# Extract the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Dataset extracted successfully.")


Dataset extracted successfully.


In [28]:
from mmengine.config import Config
import os

# Load the base config
config = Config.fromfile('./mmdetection/configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py')

# Dataset root
data_root = '/home/ronan/dataset/car_damages_new_v2/'
classes = ('crack', 'dent', 'glass shatter', 'lamp broken', 'scratch', 'tire flat')

# Update train dataset
config.train_dataloader.dataset.data_root = data_root
config.train_dataloader.dataset.ann_file = 'train/_annotations.coco.json'
config.train_dataloader.dataset.data_prefix = dict(img='train/')
config.train_dataloader.dataset.metainfo = {'classes': classes}

# Update validation dataset
config.val_dataloader.dataset.data_root = data_root
config.val_dataloader.dataset.ann_file = 'valid/_annotations.coco.json'
config.val_dataloader.dataset.data_prefix = dict(img='valid/')
config.val_dataloader.dataset.metainfo = {'classes': classes}

# Update test dataset
config.test_dataloader.dataset.data_root = data_root
config.test_dataloader.dataset.ann_file = 'test/_annotations.coco.json'
config.test_dataloader.dataset.data_prefix = dict(img='test/')
config.test_dataloader.dataset.metainfo = {'classes': classes}

# Add evaluators (important to avoid fallback to COCO val2017)
config.val_evaluator = dict(
    type='CocoMetric',
    ann_file=os.path.join(data_root, 'valid/_annotations.coco.json'),
    metric='bbox',
    format_only=False,
    classwise=True
)

config.test_evaluator = dict(
    type='CocoMetric',
    ann_file=os.path.join(data_root, 'test/_annotations.coco.json'),
    metric='bbox',
    format_only=False,
    classwise=True
)

# Update number of classes in model
config.model.roi_head.bbox_head.num_classes = len(classes)

# Training settings
config.train_cfg.max_epochs = 10  # you can change this
config.work_dir = './jedox/work_dirs/car_damage_v1'
config.default_hooks.logger.interval = 10
config.visualizer.vis_backends = [dict(type='LocalVisBackend'), dict(type='TensorboardVisBackend')]

# Save config
config.dump('./jedox/work_dirs/car_damage_v1/car_damage_config.py')

print("✅ Config updated and saved.")


✅ Config updated and saved.


In [ ]:
from mmengine.runner import Runner
from mmengine.config import Config

# Load the saved config
cfg = Config.fromfile('./jedox/work_dirs/car_damage_v1/car_damage_config.py')

# Start training
runner = Runner.from_cfg(cfg)
runner.train()


08/04 03:27:29 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]
    CUDA available: True
    numpy_random_seed: 1401853555
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.8, V12.8.93
    GCC: gcc (GCC) 11.5.0 20240719 (Red Hat 11.5.0-5)
    PyTorch: 2.1.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencod

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/ronan/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


08/04 03:27:32 - mmengine - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

08/04 03:27:32 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
08/04 03:27:32 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
08/04 03:27:32 - mmengine - INFO - Checkpoints will be saved to /home/ronan/jedox/work_dirs/car_damage_v1.
08/04 03:27:38 - mmengine - INFO - Epoch(train)  [1][  10/1473]  lr: 3.8036e-04  eta: 2:23:29  time: 0.5849  data_time: 0.0186  memory: 3882  loss: 1.9983  loss_rpn_cls: 0.6800  loss_rpn_bbox: 0.0335  loss_cls: 1.2757  acc: 99.6094  loss_bbox: 0.0091
08/04 03:27:43 - mmengine - INFO - Epoch(train)  [1][  20/1473]  lr: 7.8076e-04  eta: 2:17:59  time: 0.5628  data_time: 0.0118  memory: 3882  loss: 1.3879  loss_rpn_cls: 0.6459  loss_rpn_

In [ ]:
from mmengine.runner import Runner

# Load your config
from mmengine.config import Config
cfg = Config.fromfile('./jedox/work_dirs/car_damage_v1/car_damage_config.py')

# Update config for evaluation
cfg.load_from = './jedox/work_dirs/car_damage_v1/epoch_10.pth'
runner = Runner.from_cfg(cfg)
runner.test()


In [ ]:
from mmdet.apis import init_detector, inference_detector
import os
from glob import glob

# Paths
config_file = './jedox/work_dirs/car_damage_v1/car_damage_config.py'
checkpoint_file = './jedox/work_dirs/car_damage_v1/epoch_10.pth'
test_dir = '/home/ronan/dataset/car_damages_external_test/'
output_dir = './external_test_results'
os.makedirs(output_dir, exist_ok=True)

# Load model
model = init_detector(config_file, checkpoint_file, device='cuda:0')

# Run inference
img_paths = glob(os.path.join(test_dir, '*.jpg'))
for img_path in img_paths:
    result = inference_detector(model, img_path)
    out_path = os.path.join(output_dir, os.path.basename(img_path))
    model.show_result(img_path, result, out_file=out_path)

print("✅ Inference complete. Check output images in:", output_dir)
